In [1]:
import pandas as pd
import os
import numpy as np
import folium
import h3
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
import branca

p = os.path.join
cmap = plt.get_cmap('hot_r')
df = pd.read_csv(p('..', '03 Prediction', 'final_prediction.csv'), index_col=0)
df.time = df.time.astype(str)
RING_SIZE = 7
    
if not os.path.exists('www/maps'):
    os.mkdir('www/maps')

In [2]:
def normalize_name(city_name):
    return city_name.lower().replace(' ', '')

# Maps

In [23]:
for (c, city_df) in df.groupby('city'):
    ring = {r for h in city_df.index for r in h3.k_ring(h, 3)}
    latlons = [h3.h3_to_geo(r) for r in ring]
    lat, lon = np.array(latlons).mean(axis=0)
    
    
    for(t, time_df) in city_df.groupby('time'):
        m = folium.Map(location=[lat, lon])
        m.fit_bounds(latlons)
        cmap_legend = branca.colormap.LinearColormap([cmap(i/10) for i in range(10)], vmin=0, vmax=time_df['NO2_adjusted'].max())
        cmap_legend.caption = 'NO2 Level (ppb)'
        cmap_legend.add_to(m)

        for r in ring:
            try:
                level = time_df.loc[r]['NO2_adjusted']
            except KeyError:
                continue
            folium.Polygon(
                h3.h3_to_geo_boundary(r),
                tooltip=f'{np.round(level, 1)} ppb',
                fill_color= rgb2hex(cmap(level/time_df['NO2_adjusted'].max())), 
                color='black',
                fill_opacity= 0.6
            ).add_to(m)
        m.save(p('www','maps',f'{normalize_name(c)}_{t}.html'))

In [4]:
cities = [('index', 'Home')] + [(normalize_name(c), c) for c in df.city.unique()]
times = [""] + list(df.time.unique())

# Pages

In [29]:
from jinja2 import FileSystemLoader, Environment

def time_format(t):
    return '/'.join(reversed(t.split('-')))

def render_from_template(directory, template_name, **kwargs):
    loader = FileSystemLoader(directory)
    env = Environment(loader=loader)
    template = env.get_template(template_name)
    template.globals['time_format'] = time_format
    return template.render(**kwargs)

for (c, _) in cities:
    for t in times:
        if c == 'index' and t != '':
            continue
        name = f'{c}_{t}.html' if t else f'{c}.html'
        with open(p('www', name), 'w') as f:
            f.write(render_from_template('.', 'template.jinja', current=c, cities=cities, current_time=t, times=times))